# Get Data
Daten von der BGG-API abfragen und in einen Dataframe speichern.

In [2]:
import requests
import xml.etree.ElementTree as ET
import json
import xmltodict
import pandas as pd

# Aktuelle Hitliste laden
Hier wird die auf BGG bereitgestellte csv-Datei verwendet

In [3]:
games = pd.read_csv("../data/boardgames_ranks.csv")

In [4]:
ngames = 500
games = games.iloc[:ngames,:].copy()

In [5]:
ids = games["id"]

# API anfragen

In [6]:
def bggApiFromID(id):
    
    response = requests.get(f"https://api.geekdo.com/xmlapi/boardgame/{id}?")
    
    root = ET.fromstring(response.content)
    
    result = {}
    for boardgame in root.findall('.//boardgame'):
        for element in boardgame:
            if element.tag != "poll":
                #wenn das bereits in result, dann anfügen, sonst neu anlegen 
                if element.tag in result:
                    result[element.tag].append(element.text)
                else:
                    result[element.tag] = [element.text]

            else:
                result[element.get('name')] = []
                for child in element:
                    for subchild in child:
                        result[element.get('name')].append(subchild.attrib)

    #wenn nur ein Element drin, dann kein Array mehr draus machen
    for key in result:
        if len(result[key])==1:
            result[key] = result[key][0]

    resultdf = pd.DataFrame()
    for key in result:
        resultdf[key] = ""
        resultdf.loc[0,key] = str(result[key])

    return resultdf

In [9]:
# die ersten x ranks abfragen und in DF speichern
bgdata = pd.DataFrame()
for i in range(ngames):
    bgdata = pd.concat([bgdata, bggApiFromID(ids[i])], ignore_index=True)

In [10]:
bgdata

,yearpublished,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,age,name,description,thumbnail,...,boardgameintegration,boardgamegraphicdesigner,boardgameeditor,boardgamewriter,boardgamesculptor,boardgamesolodesigner,boardgamecompilation,boardgameinsertdesigner,boardgame,boardgameissuearticle
0,2018,2,4,120,60,120,14,"['Brass: Birmingham', 'Brass. Бирмингем', 'Bra...",Brass: Birmingham is an economic strategy game...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,2,4,60,60,60,13,"['Pandemic Legacy: Saison 1', 'Pandemic Legac...",Pandemic Legacy is a co-operative campaign gam...,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,1,4,120,60,120,14,"['Gloomhaven', 'Gloomhaven, aventures à Havren...",Gloomhaven is a game of Euro-inspired tactica...,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,...,"['Frosthaven', 'Gloomhaven: Jaws of the Lion',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,1,4,150,90,150,14,"['Archa Nova', 'Arche Nova', 'Ark Nova', 'Нови...","In Ark Nova, you will plan and design a modern...",https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,...,NaN,"['Steffen Bieker', 'Christof Tisch']","['Frank Heeren', 'Inga Keutmann', 'Bastian Win...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,3,6,480,240,480,14,"['Twilight Imperium: Čtvrtá edice', 'Twilight ...",Twilight Imperium (Fourth Edition) is a game o...,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE...,...,NaN,Shaun Boyke,NaN,"['Adam Baker', 'Daniel Clark (I)']",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2023,1,5,90,60,90,14,"['Apiario', 'Apiary', 'Včelín']","In a far-distant future, humans no longer inha...",https://cf.geekdo-images.com/dT1vJbUizZFmJAphK...,...,NaN,Christine Santana,NaN,NaN,Heriberto Valle Martinez,"['Lines J. Hutter', 'David J. Studley']",NaN,NaN,NaN,NaN
496,2016,2,4,45,45,45,14,"['Mystic Vale', 'Містична Долина', 'ミスティック・ベール']",A curse has been placed on the Valley of Life....,https://cf.geekdo-images.com/2cf8lebaIrJ8Z62HG...,...,NaN,Matt Paquette & Co.,NaN,NaN,NaN,NaN,Mystic Vale: Essential Edition,NaN,NaN,NaN
497,2023,1,5,90,60,90,14,"['Ekspedycyje', 'Expedice', 'Expedíciók', 'Exp...",The sequel to Scythe sends players on a new ad...,https://cf.geekdo-images.com/9eBww9iAi472T2goi...,...,NaN,Christine Santana,NaN,NaN,Heriberto Valle Martinez,Morten Monrad Pedersen,NaN,NaN,NaN,NaN
498,2014,2,6,40,40,40,10,"['Colt Express', 'Colt Express Η Ληστεία του Τ...","On the 11th of July, 1899 at 10 a.m., the Unio...",https://cf.geekdo-images.com/2HKX0QANk_DY7CIVK...,...,"['Colt Express: Indians & Cavalry', 'Colt Expr...",NaN,NaN,NaN,NaN,NaN,Colt Express: BIG BOX,NaN,NaN,NaN


In [11]:
games.reset_index(inplace=True, drop=True)
bgdata.reset_index(inplace=True, drop=True)

In [13]:
final = pd.concat([games,bgdata], axis=1)
final

,id,name,yearpublished,rank,bayesaverage,average,usersrated,abstracts_rank,cgs_rank,childrensgames_rank,...,boardgameintegration,boardgamegraphicdesigner,boardgameeditor,boardgamewriter,boardgamesculptor,boardgamesolodesigner,boardgamecompilation,boardgameinsertdesigner,boardgame,boardgameissuearticle
0,224517,Brass: Birmingham,2018,1,8.41691,8.59924,44539,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,161936,Pandemic Legacy: Season 1,2015,2,8.38234,8.52972,53033,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,174430,Gloomhaven,2017,3,8.35867,8.59425,61620,NaN,NaN,NaN,...,"['Frosthaven', 'Gloomhaven: Jaws of the Lion',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,342942,Ark Nova,2021,4,8.33308,8.53772,41657,NaN,NaN,NaN,...,NaN,"['Steffen Bieker', 'Christof Tisch']","['Frank Heeren', 'Inga Keutmann', 'Bastian Win...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.24250,8.60574,23298,NaN,NaN,NaN,...,NaN,Shaun Boyke,NaN,"['Adam Baker', 'Daniel Clark (I)']",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,400314,Apiary,2023,496,6.96413,7.89958,3581,NaN,NaN,NaN,...,NaN,Christine Santana,NaN,NaN,Heriberto Valle Martinez,"['Lines J. Hutter', 'David J. Studley']",NaN,NaN,NaN,NaN
496,194607,Mystic Vale,2016,497,6.96185,7.25808,11774,NaN,NaN,NaN,...,NaN,Matt Paquette & Co.,NaN,NaN,NaN,NaN,Mystic Vale: Essential Edition,NaN,NaN,NaN
497,379078,Expeditions,2023,498,6.96172,7.71018,4621,NaN,NaN,NaN,...,NaN,Christine Santana,NaN,NaN,Heriberto Valle Martinez,Morten Monrad Pedersen,NaN,NaN,NaN,NaN
498,158899,Colt Express,2014,499,6.95901,7.09476,31976,NaN,NaN,NaN,...,"['Colt Express: Indians & Cavalry', 'Colt Expr...",NaN,NaN,NaN,NaN,NaN,Colt Express: BIG BOX,NaN,NaN,NaN


In [14]:
final.to_pickle("../data/bg_data20240412.pkl")